# Dieta

## Descripción

El objetivo del problema de la dieta es seleccionar a partir de una serie de comidas, cuál es la combinación óptima de comidas que satisface los requerimientos nutricionales, incurriendo en el menor coste posible.Las comidas disponibles y su información nutricional están recogidas en el siguiente Excel:

In [ ]:
from pandas import read_excel

df = read_excel("diet.xls", nrows=64, index_col="Foods")
df

Con esto podemos definirnos un índice de comidas y otro de nutrientes (de calorías en adelante):

In [ ]:
foods = df.index.tolist()
nutrients = df.columns[1:].tolist()
print(foods) 
print(nutrients)

También conviene descomponer los costes de las comidas:

In [ ]:
food_costs = df['Price/Serving']
food_costs

De su información nutricional:

In [ ]:
food_nutrients = df[nutrients]
food_nutrients

Unas líneas más abajo el Excel también recoge las cantidades diarias mínimas y máximas deseadas de cada uno de estos nutrientes:



In [ ]:
nutrient_limits = (
    read_excel("diet.xls", skiprows=65, nrows=2, usecols=list(range(1, 2+len(nutrients))), names=["Limit"] + nutrients)
    .set_index("Limit")
    .T    # para trasponer
    .rename_axis(columns=None, index="Nutrients")
)
nutrient_limits

## Modelado del problema

Este problema se puede formular como un LP, en el que las restricciones son el número y cantidad de nutrientes:


* Índices

 $$
 \begin{array}{rcl}
 F & = & \text{comidas} \\ 
 N & = & \text{nutrientes} 
 \end{array}
 $$

* Parámetros

 $$
 \begin{array}{rcl}
 c_f & = & \text{coste por porción de comida} \quad \forall f \in F \\
 a_{fn} & = & \text{cantidad de nutriente $n$ en la comida $f$} \quad \forall f \in F, \; \forall n \in N \\  
 \underline{N}_n & = & \text{mínima cantidad de nutriente $n$} \quad \forall n \in N \\  
 \overline{N}_n & = & \text{máxima cantidad de nutriente $n$} \quad \forall n \in N \\  
 \end{array}
 $$
 
* Variables

$$
\begin{array}{rcl}
 x_f & = & \text{número de porciones de comida $f$ a consumir} \quad \forall f \in F
\end{array}
$$

### Objetivo

* Minimizar el coste total de comida

$$
\min \sum_{f \in F} c_f x_f
$$

### Restricciones

* Límite de consumo de cada nutriente:
 
 $$\underline{N}_n \leq \sum_{f \in F} a_{fn} x_f \leq \overline{N}_n \quad \forall n \in N$$

* Porciones no pueden ser negativas:  
 
 $$x_f \geq 0 \quad \forall f \in F$$

## Resolución del problema

Para resolver el problema usaremos la biblioteca <a href="https://coin-or.github.io/pulp/#">PuLP</a>, que instalamos así: 

In [ ]:
!pip install pulp
from pulp import *

PuLP funciona a través de listas y diccionarios. Por ejemplo, para crearnos las variables $x_i$ usamos [LpVariable](https://coin-or.github.io/pulp/technical/pulp.html#pulp.LpVariable) de la forma de abajo:

In [ ]:
food_vars = LpVariable.dicts("Food", indices=foods, lowBound=0, cat=LpContinuous)     # variables continuas, con cota inferior 0

Internamente `food_vars` es un diccionario de variables, formado en este caso por variables donde le antepone el prefijo especificado `"Food"` a cada una de las comidas que teníamos en `foods`. Además, los parámetros `lowBound` y `upBound` sirven para poner cotas inferiores y superiores a estas variables, y el parámetro `cat` sirve para establecer si son variables continuas, enteras o binarias (lo cual permite formular problemas LP, IP o MIP).

In [ ]:
food_vars

Lo primero es crear y especificar el tipo de problema. Para eso usamos [LpProblem](https://coin-or.github.io/pulp/technical/pulp.html#pulp.LpProblem). En este caso, como buscamos minimizar costes usaremos [LpMinimize](https://coin-or.github.io/pulp/technical/constants.html#pulp.constants.LpSenses). Como todavía no hemos especificado variables ni restricciones, la descripción sale vacía:  

In [ ]:
prob = LpProblem("Diet", LpMinimize)       # nombre identificativo del problema, y si es de tipo min o de tipo max
prob

Ya tenemos entonces que `food_vars` es el equivalente a $x_i$ en la formulación de arriba, y además ya está cubierto que $x_i \geq 0 \; \forall i$.  Ahora hay que añadir la función objetivo, que es la suma de los costes de cada comida $\sum_i c_i x_i$. Podemos usar [lpSum](https://coin-or.github.io/pulp/technical/pulp.html#pulp.lpSum) para hacer ese sumatorio, y añadírselo al problema con el operador `+=`. Ahora ya el problema es consciente de lo que se busca minimizar y de las variables que necesita ajustar para ello: 

In [ ]:
prob += lpSum([food_costs[i] * food_vars[i] for i in foods])     # at devuelve en una serie el valor que tiene para ese índice
prob

Lo único que nos falta es añadir las restricciones de nutrientes. Para cada nutriente tenemos que especificar que teniendo en cuenta cuántos nutrientes hay en cada comida (`food_nutrients`), en la dieta tiene que haber como poco su límite inferior, y no pasarse del límite superior (`nutrient_limits`). 

La forma es también a través de `lpSum` y añadiéndoselo a `prob`, sólo que lo tenemos que hacer nutriente a nutriente:

In [ ]:
for n in nutrients:
  aux = lpSum([food_nutrients[n][f] * food_vars[f] for f in foods])    # frame primero indexa por columnas, y luego por filas
  prob += (aux >= nutrient_limits["Minimum daily intake"][n])          # por lo menos esto
  prob += (aux <= nutrient_limits["Maximum daily intake"][n])          # como mucho esto

prob

Ahora resolver el problema es tan sencillo como llamar al método [solve](https://coin-or.github.io/pulp/technical/pulp.html#pulp.LpProblem.solve), y ver si ha terminado bien comprobando el [status](https://coin-or.github.io/pulp/technical/constants.html#pulp.constants.LpStatus): 

In [ ]:
status = prob.solve()
print("Status: ", LpStatus[status])

Ha terminado bien y encontrado una solución óptima. Para comprobar ahora qué comidas ha elegido, podemos gracias a la función [value](https://coin-or.github.io/pulp/technical/pulp.html#pulp.value) ir imprimiendo cuáles se han elegido:

In [ ]:
for v in prob.variables():
  val = value(v)
  if val > 0.0:     # no imprimir las comidas que no se eligen (valor 0.0)
    print(v.name, "=", v.varValue)

Lo cual supone un coste de dieta de:

In [ ]:
value(prob.objective)

## Ejercicios adicionales

1. Comprobar que, con las comidas elegidas en la solución, las restricciones de nutrientes se cumplen. ¿Hay algún nutriente para el cual el aporte de la dieta solución sea el mínimo requerido? ¿Y alguno para el que sea el máximo permitido? (**Pista: se puede invocar a `value` también con expresiones de tipo `lpSum`**).
2. Comprobar que, aunque se quiten del fichero Excel comidas no elegidas en la solución óptima, la solución al problema sigue siendo la misma (**Pista: no hace falta quitarlas del fichero en sí, sino que basta con quitarlas en el `DataFrame` utilizado**). 
3. Volviendo a usar todas las comidas, comprobar qué pasa si se impone la restricción adicional de que las raciones de comidas tienen que ser enteras (por ejemplo, no puede haber 2.29 naranjas, sino que tienen que ser o 2 o 3, sin decimales). ¿Cuál es ahora la mejor combinación? (**Pista: utilizar variables `LpInteger`**).
4. Comprobar que la nueva solución de 3 sigue cumpliendo las restricciones de nutrientes. ¿Cuáles son ahora los nutrientes para los que se aporta el mínimo/máximo estipulado? ¿Por qué el coste de esta solución es mayor que el que habíamos obtenido antes?
5. En la solución de 3 se eligen tanto kiwis como naranjas. Introducir la restricción adicional de que sólo se puede tener una de las dos frutas como mucho, es decir, o kiwis o naranjas, pero no ambas. ¿Cuál es la nueva solución ahora y su coste? (**Pista: utilizar variables adicionales `LpBinary`**).
6. Resolver el problema original (es decir, con variables `LpContinuous` y los 64 alimentos disponibles), pero mediante la función [`linprog`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html#scipy.optimize.linprog) de `scipy` en lugar de con `PuLP`. ¿Se obtiene la misma solución? (**Pista: consultar la documentación y pensar en cómo poner el problema en notación matricial $A x \leq b$**).